In [11]:
import numpy as np
import scipy.special
from scipy.signal import convolve2d

In [26]:
x_ = np.linspace(0., 1., 2)
y_ = np.linspace(1., 2., 3)

In [34]:
x, y = np.meshgrid(x_, y_, indexing='ij')

In [35]:
np.arrange